In [1]:
import pandas as pd
import exchange_calendars as ecals

import baostock as bs


from datetime import datetime, timedelta

In [2]:
csi300 = pd.read_csv('all_hs300_stocks.csv')
csi500 = pd.read_csv('all_zz500_stocks.csv')

In [3]:
def transform_code(code):
    if len(code) >= 3:
        return code[:2].upper() + code[3:]  # 前两个字符变为大写，删除第三个字符
    return code.upper()  # 如果长度不足，返回全大写

# 应用函数到 code 列
csi500['code'] = csi500['code'].apply(transform_code)
csi300['code'] = csi300['code'].apply(transform_code)

In [4]:
calendar = ecals.get_calendar('XSHG')
start_date = '2008-01-01'
end_date = '2022-12-31'
trading_days = calendar.sessions_in_range(start_date, end_date)

In [5]:
trading_days

missing_days = []
# find different between csi500 date between trading_days
csi500['date'] = pd.to_datetime(csi500['date'])
for date in trading_days:
    if date not in csi500['date'].values:
        missing_days.append(date)

In [6]:
missing_days.__len__()

27

In [7]:
hs300_stocks_missing = pd.DataFrame()
zz500_stocks_missing = pd.DataFrame()

    # 登陆系统
    
lg = bs.login()
#current_date = start_date
for current_date in missing_days:
    
    if lg.error_code != '0':
        print(f"Login failed with error: {lg.error_msg}")
    print(f"Querying date {current_date.strftime('%Y-%m-%d')}")
    date_str = current_date.strftime("%Y-%m-%d")
    rs = bs.query_hs300_stocks(date=date_str)
    if rs.error_code == '0':
        hs300_stocks = []
        while rs.next():
            stock_data = rs.get_row_data()
            stock_code = stock_data[1]  # 假设股票代码在第二列
            # 查询股票的每日信息
            k_rs = bs.query_history_k_data_plus(stock_code,
                                                    "date,code,open,close,high,low,volume",
                                                    start_date=date_str, end_date=date_str,
                                                    frequency="d", adjustflag="3")
            if k_rs.error_code == '0':
                k_data = k_rs.get_row_data()
                if k_data == []:
                    break
                else:
                        # 添加open, close, high, low, volume信息
                    hs300_stocks.append(k_data)
        if hs300_stocks:
            result = pd.DataFrame(hs300_stocks, columns=['date',
                                                             'code',
                                                             'open','close','high','low','volume'])
            hs300_stocks_missing = pd.concat([hs300_stocks_missing, result])
    else:
        print(f"Error querying date {date_str}: {rs.error_msg}")
for current_date in missing_days:
    if lg.error_code != '0':
        print(f"Login failed with error: {lg.error_msg}")
    print(f"Querying date {current_date.strftime('%Y-%m-%d')}")
    date_str = current_date.strftime("%Y-%m-%d")
    rs = bs.query_zz500_stocks(date=date_str)
    if rs.error_code == '0':
        hs300_stocks = []
        while rs.next():
            stock_data = rs.get_row_data()
            stock_code = stock_data[1]  # 假设股票代码在第二列
            # 查询股票的每日信息
            k_rs = bs.query_history_k_data_plus(stock_code,
                                                    "date,code,open,close,high,low,volume",
                                                    start_date=date_str, end_date=date_str,
                                                    frequency="d", adjustflag="3")
            if k_rs.error_code == '0':
                k_data = k_rs.get_row_data()
                if k_data != []:
                  hs300_stocks.append(k_data)
        if hs300_stocks:
            result = pd.DataFrame(hs300_stocks, columns=['date',
                                                             'code',
                                                             'open','close','high','low','volume'])
            zz500_stocks_missing = pd.concat([zz500_stocks_missing, result])
    else:
        print(f"Error querying date {date_str}: {rs.error_msg}")
print("all done")

login success!
Querying date 2008-06-19
Querying date 2009-03-03
Querying date 2009-08-21
Querying date 2010-02-08
Querying date 2010-05-05
Querying date 2011-04-12
Querying date 2011-07-07
Querying date 2012-06-13
Querying date 2012-09-07
Querying date 2013-02-25
Querying date 2013-05-22
Querying date 2014-04-29
Querying date 2014-07-24
Querying date 2015-07-01
Querying date 2015-09-25
Querying date 2016-03-14
Querying date 2016-09-01
Querying date 2017-05-16
Querying date 2017-11-03
Querying date 2018-07-18
Querying date 2018-10-12
Querying date 2019-04-01
Querying date 2019-09-19
Querying date 2020-06-02
Querying date 2020-11-20
Querying date 2021-08-04
Querying date 2022-04-18
Querying date 2008-06-19
Querying date 2009-03-03
Querying date 2009-08-21
Querying date 2010-02-08
Querying date 2010-05-05
Querying date 2011-04-12
Querying date 2011-07-07
Querying date 2012-06-13
Querying date 2012-09-07
Querying date 2013-02-25
Querying date 2013-05-22
Querying date 2014-04-29
Querying d

In [8]:
hs300_stocks_missing['code'] = hs300_stocks_missing['code'].apply(transform_code)
zz500_stocks_missing['code'] = zz500_stocks_missing['code'].apply(transform_code)

In [9]:
csi300_complete = pd.concat([hs300_stocks_missing, csi300])
csi500_complete = pd.concat([zz500_stocks_missing, csi500])

In [10]:
csi300_complete.set_index(['date', 'code'], inplace=True)


In [11]:
csi300_complete.reset_index(inplace=True)

In [12]:
csi500_complete

,date,code,open,close,high,low,volume
0,2008-06-19,SH600039,7.0800,6.5300,7.1000,6.5300,12491849
1,2008-06-19,SH600051,6.1000,6.2500,6.4100,6.0400,7504955
2,2008-06-19,SH600054,14.6800,13.6100,14.8500,13.5100,1049794
3,2008-06-19,SH600056,11.4000,10.5400,11.6500,10.5400,1608679
4,2008-06-19,SH600057,2.9900,2.9500,2.9900,2.9500,1802300
...,...,...,...,...,...,...,...
1810948,2022-12-30 00:00:00,SZ300741,23.3,23.53,23.59,23.25,697539.0
1810949,2022-12-30 00:00:00,SZ300769,232.32,229.59,239.98,226.5,2917928.0
1810950,2022-12-30 00:00:00,SZ300832,49.31,50.14,50.52,48.47,3582705.0
1810951,2022-12-30 00:00:00,SZ300869,32.06,32.57,33.68,31.1,23415864.0


In [13]:
csi500_complete

,date,code,open,close,high,low,volume
0,2008-06-19,SH600039,7.0800,6.5300,7.1000,6.5300,12491849
1,2008-06-19,SH600051,6.1000,6.2500,6.4100,6.0400,7504955
2,2008-06-19,SH600054,14.6800,13.6100,14.8500,13.5100,1049794
3,2008-06-19,SH600056,11.4000,10.5400,11.6500,10.5400,1608679
4,2008-06-19,SH600057,2.9900,2.9500,2.9900,2.9500,1802300
...,...,...,...,...,...,...,...
1810948,2022-12-30 00:00:00,SZ300741,23.3,23.53,23.59,23.25,697539.0
1810949,2022-12-30 00:00:00,SZ300769,232.32,229.59,239.98,226.5,2917928.0
1810950,2022-12-30 00:00:00,SZ300832,49.31,50.14,50.52,48.47,3582705.0
1810951,2022-12-30 00:00:00,SZ300869,32.06,32.57,33.68,31.1,23415864.0


In [14]:
csi300_complete['date'] = pd.to_datetime(csi300_complete['date'])
csi500_complete['date'] = pd.to_datetime(csi500_complete['date'])

In [15]:
csi500_complete

,date,code,open,close,high,low,volume
0,2008-06-19,SH600039,7.0800,6.5300,7.1000,6.5300,12491849
1,2008-06-19,SH600051,6.1000,6.2500,6.4100,6.0400,7504955
2,2008-06-19,SH600054,14.6800,13.6100,14.8500,13.5100,1049794
3,2008-06-19,SH600056,11.4000,10.5400,11.6500,10.5400,1608679
4,2008-06-19,SH600057,2.9900,2.9500,2.9900,2.9500,1802300
...,...,...,...,...,...,...,...
1810948,2022-12-30,SZ300741,23.3,23.53,23.59,23.25,697539.0
1810949,2022-12-30,SZ300769,232.32,229.59,239.98,226.5,2917928.0
1810950,2022-12-30,SZ300832,49.31,50.14,50.52,48.47,3582705.0
1810951,2022-12-30,SZ300869,32.06,32.57,33.68,31.1,23415864.0


In [16]:
def custom_sort(code):
    # 检查是否以 'SH' 开头
    return (code[:2] != 'SH', int(code[2:]))  # SH 优先，后面数字大小排序

# 按照 date 分组并排序
csi500_complete = (
    csi500_complete.assign(sort_key=csi500_complete['code'].apply(custom_sort))  # 添加排序关键字
      .sort_values(by=['date', 'sort_key'])  # 先按 date 排序，然后按 sort_key 排序
      .drop(columns='sort_key')  # 删除临时的排序关键字列
)

csi300_complete = (
    csi300_complete.assign(sort_key=csi300_complete['code'].apply(custom_sort))  # 添加排序关键字
      .sort_values(by=['date', 'sort_key'])  # 先按 date 排序，然后按 sort_key 排序
      .drop(columns='sort_key')  # 删除临时的排序关键字列
)

In [17]:
csi500_complete['date'].is_monotonic

/tmp/ipykernel_55074/4174939886.py:1: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  csi500_complete['date'].is_monotonic


True

In [56]:
csi300_complete.to_csv('all_hs300_stocks_v2.csv', index=False)
csi500_complete.to_csv('all_zz500_stocks_v2.csv',index=False)

In [18]:
csi800 = pd.concat([csi300_complete,csi500_complete])
csi800

,date,code,open,close,high,low,volume
8100,2008-01-02,SH600000,53.0,53.55,55.05,51.81,13158390.0
8101,2008-01-02,SH600001,8.12,8.38,8.6,7.95,62828328.0
8102,2008-01-02,SH600003,6.21,6.21,6.21,6.21,0.0
8103,2008-01-02,SH600004,20.95,22.15,22.2,20.95,10329598.0
8104,2008-01-02,SH600005,19.79,19.3,19.89,19.0,26984436.0
...,...,...,...,...,...,...,...
1810948,2022-12-30,SZ300741,23.3,23.53,23.59,23.25,697539.0
1810949,2022-12-30,SZ300769,232.32,229.59,239.98,226.5,2917928.0
1810950,2022-12-30,SZ300832,49.31,50.14,50.52,48.47,3582705.0
1810951,2022-12-30,SZ300869,32.06,32.57,33.68,31.1,23415864.0


In [19]:
csi800['date'].is_monotonic

/tmp/ipykernel_55074/3458764868.py:1: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  csi800['date'].is_monotonic


False

In [20]:
csi800_complete = (
    csi800.assign(sort_key=csi800['code'].apply(custom_sort))  # 添加排序关键字
      .sort_values(by=['date', 'sort_key'])  # 先按 date 排序，然后按 sort_key 排序
      .drop(columns='sort_key')  # 删除临时的排序关键字列
)

In [21]:
csi800_complete

,date,code,open,close,high,low,volume
8100,2008-01-02,SH600000,53.0,53.55,55.05,51.81,13158390.0
8101,2008-01-02,SH600001,8.12,8.38,8.6,7.95,62828328.0
8102,2008-01-02,SH600003,6.21,6.21,6.21,6.21,0.0
8103,2008-01-02,SH600004,20.95,22.15,22.2,20.95,10329598.0
8104,2008-01-02,SH600005,19.79,19.3,19.89,19.0,26984436.0
...,...,...,...,...,...,...,...
1094668,2022-12-30,SZ300896,568.09,566.35,570.6,557.0,1286736.0
1094669,2022-12-30,SZ300919,68.5,65.61,69.12,65.61,7426278.0
1094670,2022-12-30,SZ300957,151.0,149.24,151.7,146.89,3534444.0
1094671,2022-12-30,SZ300979,57.53,57.11,58.0,56.63,1839075.0


In [22]:
csi800_complete.to_csv("csi800_stocks_v2.csv",index=False)

In [14]:
csi300['date'].unique().__len__()

3622

In [15]:
csi500['date'].unique().__len__()

3622

In [4]:
csi_500new = pd.read_csv("all_zz500_stocks.csv")

csi_500new

,date,code,open,close,high,low,volume
0,2008-01-02,sh.600039,9.50,9.61,9.79,9.35,7202611.0
1,2008-01-02,sh.600051,10.42,11.28,11.28,10.42,5915286.0
2,2008-01-02,sh.600054,31.67,32.55,33.39,31.63,2404624.0
3,2008-01-02,sh.600056,24.60,26.69,26.69,24.60,6301816.0
4,2008-01-02,sh.600057,6.37,6.47,6.53,6.19,9258017.0
...,...,...,...,...,...,...,...
1803821,2022-12-30,sz.300741,23.30,23.53,23.59,23.25,697539.0
1803822,2022-12-30,sz.300769,232.32,229.59,239.98,226.50,2917928.0
1803823,2022-12-30,sz.300832,49.31,50.14,50.52,48.47,3582705.0
1803824,2022-12-30,sz.300869,32.06,32.57,33.68,31.10,23415864.0


In [80]:
import baostock as bs
import pandas as pd
from multiprocessing import Pool

# 定义查询每个时间区间数据的函数
def query_hs300_stocks_in_range(date_list):
    all_hs300_stocks = pd.DataFrame()
    lg = bs.login()
    # 登陆系统
    if lg.error_code != '0':
        print(f"Login failed with error: {lg.error_msg}")
        return pd.DataFrame()

    for current_date in date_list:
        print(f"Querying date {current_date}")
        rs = bs.query_zz500_stocks(date=current_date)
        if rs.error_code == '0':
            hs300_stocks = []
            while rs.next():
                stock_data = rs.get_row_data()
                stock_code = stock_data[1]  # 假设股票代码在第二列
                # 查询股票的每日信息
                k_rs = bs.query_history_k_data_plus(stock_code,
                                                    "date,code,open,close,high,low,volume",
                                                    start_date=current_date, end_date=current_date,
                                                    frequency="d", adjustflag="3")
                if k_rs.error_code == '0':
                    k_data = k_rs.get_row_data()
                    if k_data == []:
                        break
                    else:
                        # 添加open, close, high, low, volume信息
                        hs300_stocks.append(k_data)
            if hs300_stocks:
                result = pd.DataFrame(hs300_stocks, columns=['date',
                                                             'code',
                                                             'open','close','high','low','volume'])
                all_hs300_stocks = pd.concat([all_hs300_stocks, result])
        else:
            print(f"Error querying date {current_date}: {rs.error_msg}")

    # 登出系统
    #bs.logout()
    return all_hs300_stocks

# 将日期列表划分为多个子列表
def split_date_list(date_list, n_splits):
    split_size = len(date_list) // n_splits
    return [date_list[i:i + split_size] for i in range(0, len(date_list), split_size)]

if __name__ == "__main__":
    # 假设你有一个不连续的日期列表
    date_list = csi500_not_500.index

    # 将日期列表划分为多个子列表，假设我们分为4部分
    n_splits = 10
    date_sublists = split_date_list(date_list, n_splits)

    # 使用多进程并行查询每个时间子列表的数据
    with Pool(processes=n_splits) as pool:
        results = pool.map(query_hs300_stocks_in_range, date_sublists)

    # 合并所有进程的结果
    all_zz500_stocks_not500 = pd.concat(results)


    print("Data extraction completed.")


login success!login success!login success!login success!
login success!
login success!

Querying date 2019-01-07Querying date 2019-01-11
Querying date 2009-08-28Querying date 2014-06-26


Querying date 2018-12-28

Querying date 2009-05-06

login success!
Querying date 2019-01-17
login success!
Querying date 2012-03-21
login success!
Querying date 2021-09-15
Querying date 2019-01-02
login success!
Querying date 2012-02-29
Querying date 2019-01-03
Querying date 2019-01-04
Querying date 2012-03-01
Querying date 2012-03-22
login success!
Querying date 2021-09-23
Querying date 2010-02-12
Querying date 2012-03-02
Querying date 2009-05-07
Querying date 2012-03-20
Querying date 2019-01-18
Querying date 2012-03-23
Querying date 2019-01-14
Querying date 2011-02-18
Querying date 2019-01-08
Querying date 2012-02-28
login success!
Querying date 2021-09-29
Querying date 2017-01-18
Querying date 2021-09-16
Querying date 2013-04-26
Querying date 2009-05-08
Querying date 2021-09-13
Querying date 2021-0

In [84]:
all_zz500_stocks_not500.groupby('date')['code'].count()

date
2009-05-06    340
2009-05-07    340
2009-05-08    340
2009-08-27    264
2009-08-28    264
2010-02-12    180
2011-02-18    149
2012-02-28     24
2012-02-29     24
2012-03-01     24
2012-03-02     24
2012-03-20    252
2012-03-21    252
2012-03-22    252
2012-03-23    252
2013-04-26    287
2014-06-26    500
2017-01-18    296
2017-01-19    296
2017-01-20    296
2018-12-28     48
2019-01-02     48
2019-01-03     48
2019-01-04     48
2019-01-07    499
2019-01-08    499
2019-01-09    499
2019-01-10    499
2019-01-11    499
2019-01-14    499
2019-01-15    499
2019-01-16    499
2019-01-17    499
2019-01-18    499
2021-09-13    499
2021-09-14    499
2021-09-15    499
2021-09-16    499
2021-09-17    499
2021-09-22    499
2021-09-23    499
2021-09-24    499
2021-09-27    499
2021-09-28    499
2021-09-29    499
2021-09-30    499
Name: code, dtype: int64

In [101]:
bs.login()

rs = bs.query_zz500_stocks(date='2009-05-06')

stock_code =  rs.get_data().iloc[340,1]

k_rs = bs.query_history_k_data_plus(stock_code,
                                                     "date,code,open,close,high,low,volume",
                                                     start_date='2009-05-06', end_date='2009-05-06',
                                                     frequency="d", adjustflag="3")

print(k_rs.get_data())

login success!
Empty DataFrame
Columns: []
Index: []


In [4]:
csi500['date'] = pd.to_datetime(csi500['date'])
csi500.set_index(['date'], inplace=True)
csi500

# 获取数据集中最早和最晚的日期
start_date = '2008-01-01'
end_date = '2022-12-31'

# 生成完整的日期范围，假设工作日是频率
complete_date_range = pd.date_range(start=start_date, end=end_date, freq='D')  # freq='B' 表示只包含工作日

# 获取实际存在的日期
actual_dates = csi500.index.unique()

# 找出缺失的日期
missing_dates = complete_date_range.difference(actual_dates)

# 打印缺失的日期
print("Missing dates:")
print(missing_dates)

Missing dates:
DatetimeIndex(['2008-01-01', '2008-01-05', '2008-01-06', '2008-01-12',
               '2008-01-13', '2008-01-19', '2008-01-20', '2008-01-26',
               '2008-01-27', '2008-02-02',
               ...
               '2022-11-27', '2022-12-03', '2022-12-04', '2022-12-10',
               '2022-12-11', '2022-12-17', '2022-12-18', '2022-12-24',
               '2022-12-25', '2022-12-31'],
              dtype='datetime64[ns]', length=1857, freq=None)


In [29]:
import chinese_calendar as calendar
ls = []
# 找出缺失的日期中的工作日
for date in missing_dates:
    if calendar.is_workday(date):
        ls.append(date)

In [30]:
ls

[Timestamp('2008-02-02 00:00:00'),
 Timestamp('2008-02-03 00:00:00'),
 Timestamp('2008-05-04 00:00:00'),
 Timestamp('2008-06-19 00:00:00'),
 Timestamp('2008-09-27 00:00:00'),
 Timestamp('2008-09-28 00:00:00'),
 Timestamp('2009-01-04 00:00:00'),
 Timestamp('2009-01-24 00:00:00'),
 Timestamp('2009-02-01 00:00:00'),
 Timestamp('2009-03-03 00:00:00'),
 Timestamp('2009-05-31 00:00:00'),
 Timestamp('2009-08-21 00:00:00'),
 Timestamp('2009-09-27 00:00:00'),
 Timestamp('2009-10-10 00:00:00'),
 Timestamp('2010-02-08 00:00:00'),
 Timestamp('2010-02-20 00:00:00'),
 Timestamp('2010-02-21 00:00:00'),
 Timestamp('2010-05-05 00:00:00'),
 Timestamp('2010-06-12 00:00:00'),
 Timestamp('2010-06-13 00:00:00'),
 Timestamp('2010-09-19 00:00:00'),
 Timestamp('2010-09-25 00:00:00'),
 Timestamp('2010-09-26 00:00:00'),
 Timestamp('2010-10-09 00:00:00'),
 Timestamp('2011-01-30 00:00:00'),
 Timestamp('2011-02-12 00:00:00'),
 Timestamp('2011-04-02 00:00:00'),
 Timestamp('2011-04-12 00:00:00'),
 Timestamp('2011-07-

In [9]:
import baostock as bs

not_connect_date = []
bs.login()
for date in missing_dates:
    date = date.strftime("%Y-%m-%d")
    print(date)

    rs = bs.query_zz500_stocks(date=date)
    if rs.error_code == '0':
        hs300_stocks = []
        while rs.next():
            stock_data = rs.get_row_data()
            stock_code = stock_data[1]  # 假设股票代码在第二列
            # 查询股票的每日信息
            k_rs = bs.query_history_k_data_plus(stock_code,
                                                    "date,code,open,close,high,low,volume",
                                                    start_date=date, end_date=date,
                                                    frequency="d", adjustflag="3")
            if k_rs.error_code == '0':
                k_data = k_rs.get_row_data()
                if k_data == []:
                    break
                else:
                    # 添加open, close, high, low, volume信息
                    not_connect_date.append(date)
                    break
bs.logout()                    


login success!
2008-01-01
2008-01-05
2008-01-06
2008-01-12
2008-01-13
2008-01-19
2008-01-20
2008-01-26
2008-01-27
2008-02-02
2008-02-03
2008-02-06
2008-02-07
2008-02-08
2008-02-09
2008-02-10
2008-02-11
2008-02-12
2008-02-16
2008-02-17
2008-02-23
2008-02-24
2008-03-01
2008-03-02
2008-03-08
2008-03-09
2008-03-15
2008-03-16
2008-03-22
2008-03-23
2008-03-29
2008-03-30
2008-04-04
2008-04-05
2008-04-06
2008-04-12
2008-04-13
2008-04-19
2008-04-20
2008-04-26
2008-04-27
2008-05-01
2008-05-02
2008-05-03
2008-05-04
2008-05-10
2008-05-11
2008-05-17
2008-05-18
2008-05-24
2008-05-25
2008-05-31
2008-06-01
2008-06-07
2008-06-08
2008-06-09
2008-06-14
2008-06-15
2008-06-19
2008-06-21
2008-06-22
2008-06-28
2008-06-29
2008-07-05
2008-07-06
2008-07-12
2008-07-13
2008-07-19
2008-07-20
2008-07-26
2008-07-27
2008-08-02
2008-08-03
2008-08-09
2008-08-10
2008-08-16
2008-08-17
2008-08-23
2008-08-24
2008-08-30
2008-08-31
2008-09-06
2008-09-07
2008-09-13
2008-09-14
2008-09-15
2008-09-20
2008-09-21
2008-09-27
2008-0

KeyboardInterrupt: 